In [13]:
import es_core_news_sm
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import regex
import pandas as pd
from scipy.spatial.distance import cosine
import os
import nltk

In [14]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

True

In [83]:
# Funcion para entrenar el modelo
def EntrenarModelo(oraciones,NombreModelo):
    # vector_size parece establizar los resultados a partir de 100, por lo que empleamos un size=150 para darle un margen de error a dicha observacion
    model = Word2Vec(oraciones,vector_size=150, window=50, min_count=1)
    model.save(NombreModelo)
# Funcion para cargar un modelo ya existente
def CargarModelo(NombreModelo):
   modelo = Word2Vec.load(NombreModelo)
   vocabulario = [term for term in modelo.wv.key_to_index]  
   return(modelo,vocabulario)

In [84]:
# Obtener el emebdding de un TEXTO
def ObtenerEmbeddingOracion(modelo, oracion):
   Lista_vectores = []
   for w in nltk.word_tokenize(oracion, language="spanish"):
       # Verificar que la oracion w exista en el modelo
       try:
           modelo.wv[w]
       except KeyError:
           continue
       # Obtener vector de la oracion
       vec = modelo.wv[w]
       Lista_vectores.append(vec)
   embedding_palabras = np.array(Lista_vectores)
   if (len(embedding_palabras) > 0):
        embedding_oracion = embedding_palabras.mean(axis=0)
   else:
        embedding_oracion = np.zeros(modelo.vector_size)
   return(embedding_oracion) 

In [85]:
# Funcion para crear el corpus
def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="UTF-8").read()
     corpus.append(texto)
     doc_id.append(filename)
  return(corpus,doc_id)
# Preprocesamiento de textos
def PreProcesarTextos(textos):
    texto_limpio = []
    # Para cada texto tokenizar por oraciones
    for texto in textos:
        if len(texto) != 0:
            texto_limpio.append(nltk.word_tokenize(texto, language="spanish"))
    return texto_limpio
def PreProcesarTextos2(textos):
    texto_limpio = []
    # Para cada texto tokenizar por oraciones
    for texto in textos:
        if len(texto) != 0:
            texto_limpio.append(nltk.sent_tokenize(texto, language="spanish"))
    return texto_limpio

In [86]:
def CrearDiccionario(lista,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista[v]
   return(dicc)

In [87]:
PATH = "DiscursosOriginales/"
# Creamos el corpus a partir de todos los textos en el directiorio PATH
corpus,docID = CrearCorpus(PATH)
# Se divide cada texto en oraciones
oraciones = PreProcesarTextos(corpus)
CorpusConClave  = CrearDiccionario(corpus,docID)


In [88]:
# Entrenamos el modelo con los textos tokenizados por oracion y lo llamamos mi_word2vec
EntrenarModelo(oraciones,'mi_word2vec')

In [89]:
# cargamos el modelo y el vocabulario
modelo, vocabulario = CargarModelo('mi_word2vec')

In [90]:
doc1 = CorpusConClave['85792.txt']
doc2 = CorpusConClave['86506.txt']
vec1 = ObtenerEmbeddingOracion(modelo, doc1)
vec2 = ObtenerEmbeddingOracion(modelo, doc2)

similitud = 1-cosine(vec1,vec2)
print(similitud)

0.7110172510147095


In [91]:
# Ordenar oraciones por puntaje
def centroide(modelo, oraciones):
    embedding = []
    for oracion in oraciones:
        embedding.append(ObtenerEmbeddingOracion(modelo, oracion))
    return np.mean(embedding, axis=0)
def OrdenarOraciones(oraciones):
    puntaje = []
    centroidee = centroide(modelo, oraciones)
    oraciones.sort(key=lambda oracion: cosine(ObtenerEmbeddingOracion(modelo, oracion), centroidee))
    return oraciones

In [92]:
# Metodo summarization imlementado en python
def generar_resumen(O,N,P,U):
    O = OrdenarOraciones(O)
    largo = 1
    Resumen = [""]*(N+1)
    M = len(O)
    for i in range(M):
        if (largo > N): return Resumen[1:N+1]
        Vo = ObtenerEmbeddingOracion(modelo, O[i])
        incluirOracion = False
        for j in range(largo):
            try:
                Vr = ObtenerEmbeddingOracion(modelo,Resumen[j])
            except KeyError:
                Vr = np.zeros(len(Vo))
            Sim = 1-cosine(Vo, Vr)
            if Sim > U and (O[i] not in Resumen):
                incluirOracion = True
        if incluirOracion == True:
            Resumen[largo] = O[i]
            largo+=1
    return Resumen[1:N+1]

In [94]:
# Generamos un resumen de un texto aleatorio
# https://www.bacheletpresidente.cl/discursos/primer-discurso-segundo-mandato/
text = open('new.txt', encoding='UTF-8').read()
oraciones = PreProcesarTextos2([text])[0]
resumen = generar_resumen(oraciones, 10, 0,0.2)
resumen

['Sólo juntos podemos dar poder a lo local, dar voz a las diferentes necesidades de nuestra gente, dignifcar el trabajo y la democracia.',
 'Yo soy hija de la educación pública, y mi compromiso es que en Chile todos tengamos esas mismas oportunidades.',
 '¡Un programa que compromete una gran Reforma Tributaria, que consagrará el principio de que quienes tienen más, contribuyan con más al bienestar de todas y todos!',
 'Y también de un país que trabaje incansablemente para que la dignidad y el respeto de los derechos sean la regla para todos y todas.',
 '¡Es hora de iniciar juntos ese camino hacia una nación desarrollada y justa, moderna y tolerante, próspera e inclusiva que todos nos merecemos!',
 'Porque tenemos urgencia, debemos trabajar con unidad, con generosidad y con compromiso, no por los intereses propios, sino por el bien común.',
 '¡Un programa que se compromete con el medio ambiente, con ciudades y barrios amables, con regiones que sean protagonistas de su propio desarrollo!